# Hands on Machine Learning


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://atenea.upc.edu/course/view.php?id=71605" target="_blank">
      <img src="https://github.com/dmaluenda/hands_on_machine_learning/raw/master/resources/upc_logo_49px.png" width="130"/>
    </a>
  </td>
  <td>
  </td>
  <td>   <!-- gColab -->
    <a href="https://colab.research.google.com/github/dmaluenda/hands_on_machine_learning/blob/master/01_Basics_NeuralNetworks.ipynb" target="_blank">
      <img src="https://raw.githubusercontent.com/dmaluenda/hands_on_machine_learning/master/resources/colab_logo_32px.png" />
      Run in Google Colab
    </a>
  </td>
  <td>   <!-- github -->
    <a href="https://github.com/dmaluenda/hands_on_machine_learning/blob/master/01_Basics_NeuralNetworks.ipynb" target="_blank">
      <img src="https://raw.githubusercontent.com/dmaluenda/hands_on_machine_learning/master/resources/github_logo_32px.png" />
      View source on GitHub
    </a>
  </td>
  <td>   <!-- download -->
    <a href="https://raw.githubusercontent.com/dmaluenda/hands_on_machine_learning/master/01_Basics_NeuralNetworks.ipynb"  target="_blank" download="01_Basics_NeuralNetworks">
      <img src="https://raw.githubusercontent.com/dmaluenda/hands_on_machine_learning/master/resources/download_logo_32px.png" />
      Download notebook
      </a>
  </td>
</table>

# $\text{I}$. Neural Networks with Pure Python

Hands on "Machine Learning on Classical and Quantum data" course of
[Master in Photonics - PHOTONICS BCN](https://photonics.masters.upc.edu/en/general-information)
[[UPC](https://photonics.masters.upc.edu/en) +
[UB](https://www.ub.edu/web/ub/en/estudis/oferta_formativa/master_universitari/fitxa/P/M0D0H/index.html?) +
[UAB](https://www.uab.cat/en/uab-official-masters-degrees-study-guides/) +
[ICFO](https://www.icfo.eu/lang/studies/master-studies)].

Tutorial 1

This notebook shows how to:
- implement the forward-pass (evaluation) of a deep, fully connected neural network in a few lines of python
- do that efficiently using batches
- illustrate the results for randomly initialized neural networks

**References**:

[1] [Machine Learning for Physicists](https://machine-learning-for-physicists.org/) by Florian Marquardt.<br>
[2] [NumPy](https://numpy.org/doc/stable/user/whatisnumpy.html): the fundamental package for scientific computing in Python.<br>
[3] [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html): a comprehensive library for creating static, animated, and interactive visualizations in Python.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports:-only-numpy-and-matplotlib" data-toc-modified-id="Imports:-only-numpy-and-matplotlib-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Imports: only numpy and matplotlib</a></span></li><li><span><a href="#A-very-simple-neural-network-(no-hidden-layer)" data-toc-modified-id="A-very-simple-neural-network-(no-hidden-layer)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A very simple neural network (no hidden layer)</a></span><ul class="toc-item"><li><span><a href="#Simple-implementation-(single-input)" data-toc-modified-id="Simple-implementation-(single-input)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simple implementation (single input)</a></span></li><li><span><a href="#More-compact" data-toc-modified-id="More-compact-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>More compact</a></span></li><li><span><a href="#Multiple-inputs" data-toc-modified-id="Multiple-inputs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Multiple inputs</a></span></li></ul></li><li><span><a href="#NN-with-one-hidden-layer" data-toc-modified-id="NN-with-one-hidden-layer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>NN with one hidden layer</a></span></li><li><span><a href="#'batch'-processing-of-Neural-Networks" data-toc-modified-id="'batch'-processing-of-Neural-Networks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>'batch' processing of Neural Networks</a></span><ul class="toc-item"><li><span><a href="#Matrix-vector-multiplication-of-array-dimensions" data-toc-modified-id="Matrix-vector-multiplication-of-array-dimensions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Matrix-vector multiplication of array dimensions</a></span></li><li><span><a href="#Defining-functions-to-evaluate-a-layer-and-a-network-with-batch-processing" data-toc-modified-id="Defining-functions-to-evaluate-a-layer-and-a-network-with-batch-processing-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Defining functions to evaluate a layer and a network with batch processing</a></span></li><li><span><a href="#Now-visualize-this-multi-layer-net,-now-more-efficiently!" data-toc-modified-id="Now-visualize-this-multi-layer-net,-now-more-efficiently!-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Now visualize this multi-layer net, now more efficiently!</a></span></li><li><span><a href="#A-network-with-MANY-hidden-layers-(same-size-each)" data-toc-modified-id="A-network-with-MANY-hidden-layers-(same-size-each)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>A network with MANY hidden layers (same size each)</a></span></li><li><span><a href="#[Homework]-Try-with-another-nonlinear-activation-function" data-toc-modified-id="[Homework]-Try-with-another-nonlinear-activation-function-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>[Homework] Try with another nonlinear activation function</a></span></li></ul></li><li><span><a href="#Summing-up-in-general-functions" data-toc-modified-id="Summing-up-in-general-functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Summing up in general functions</a></span></li><li><span><a href="#Fancy-visualization-of-Neural-Networks-with-Pure-Python" data-toc-modified-id="Fancy-visualization-of-Neural-Networks-with-Pure-Python-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fancy visualization of Neural Networks with Pure Python</a></span><ul class="toc-item"><li><span><a href="#Some-internal-routines-for-plotting-the-network" data-toc-modified-id="Some-internal-routines-for-plotting-the-network-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Some internal routines for plotting the network</a></span></li><li><span><a href="#No-hidden-layer-NN" data-toc-modified-id="No-hidden-layer-NN-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>No hidden layer NN</a></span></li><li><span><a href="#Deep-dense-NN" data-toc-modified-id="Deep-dense-NN-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Deep dense NN</a></span></li><li><span><a href="#Something-not-random" data-toc-modified-id="Something-not-random-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Something not random</a></span></li></ul></li></ul></div>

## Imports: only numpy and matplotlib

In [ ]:
# get the "numpy" library for linear algebra
import numpy as np

# get "matplotlib" for plotting
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300  # highres display
from matplotlib.axes._axes import _log as mpl_ax_logger
mpl_ax_logger.setLevel('ERROR')  # ignore warnings


# for nice inset colorbars:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# time control to count it and manage it
from time import time

## A very simple neural network (no hidden layer)

A network with $N_0$ input neurons and $N_1$ output neurons (no hidden layer)

$$y^{\rm out}_j = f(\sum_k^{N_0} w_{jk} y^{\rm in}_k + b_j) \quad ; \quad j=1\dots N_1$$

where $w$ is the weight matrix, $b$ is the bias vector, and $f$ would be the activation function (e.g. the sigmoid here), which is applied independently for each $j$.

### Simple implementation (single input)

In [ ]:
N0 = 3  # input layer size
N1 = 2  # output layer size

# initialize random weights: array dimensions N1xN0
w = np.random.uniform(low=-1, high=+1, size=(N1,N0))

# initialize random biases: N1 vector
b = np.random.uniform(low=-1, high=+1, size=N1)

Let's start with an arbitrary input:

In [ ]:
# input values
y_in = np.array([0.2, 0.4, -0.1])

In [ ]:
# evaluate network by hand, in two steps
z = np.dot(w, y_in) + b  # result: the vector of 'z' values, length N1
y_out = 1 / (1 + np.exp(-z))  # the 'sigmoid' function (applied elementwise)

In [ ]:
print("network input y_in:", y_in)
print("weights w:", w)
print("bias vector b:", b)
print("linear superposition z:", z)
print("network output y_out:", y_out)

### More compact

Still stay with the simple network, but define a function that evaluates the network, and visualize the  output for various inputs

In [ ]:
# a function that applies the network
def apply_basic_net(y_in):
    global w, b  # let's use global variables. We could pass them as arguments
    
    z = np.dot(w, y_in) + b    
    return 1 / (1 + np.exp(-z))


Let's play with a different number of neurons

**NOTE**: NNs with 2 input neurons and 1 output neuron are ideal to learn how it
works because it allows us to visualize its behavior in a single picture. Let's see

In [ ]:
N0 = 2  # input layer size
N1 = 1  # output layer size

w = np.random.uniform(low=-10, high=+10, size=(N1,N0))  # random weights: N1xN0
b = np.random.uniform(low=-1, high=+1, size=N1)  # biases: N1 vector

Define a valid input array

In [ ]:
y_in = []  # Fill this array 
apply_basic_net(y_in)  # a simple test with an arbitrary input

Which shape y_in must be? And the output? Why?

### Multiple inputs

Let's see how the network acts with many different inputs

In [ ]:
M = 50  # Number of inputs to explore
y_out = np.zeros([M, M]) # array MxM, to hold the result

for j1 in range(M): 
    for j2 in range(M):
        # out of these integer indices, generate
        # two values in the range -0.5...0.5
        # and then apply the network to those two
        # values, being the input
        value0 = float(j1) / M - 0.5
        value1 = float(j2) / M - 0.5
        y_in = [value0, value1]
        y_out[j1, j2] = apply_basic_net(y_in)

**Note**: this is NOT the most efficient way to do this! (but simple). We will later learn how to use array syntax efficiently

The output is an array (50 $\times$ 50) containing all combinations of values from -0.5 to 0.5 

In [ ]:
# display image
plt.imshow(y_out, origin='lower', extent=(-0.5,0.5,-0.5,0.5))
plt.colorbar()
plt.title("NN output as a function of input values")
plt.xlabel("y_2")
plt.ylabel("y_1")
plt.show()

What represent this image? Which kind of image is this?

## NN with one hidden layer

The idea here is to have multiple weight matrices (for each pair of subsequent layers there is one weight matrix). The function that "applies a layer", i.e. goes from one layer to the next, is essentially the same as the function evaluating the simple network above. 

In [ ]:
def apply_basic_layer(y_in, w, b):
    """ A function that evaluates one layer based on 
        the neuron values in the preceding layer.
        Now, we pass w and b as argument to be able 
        to accommodate them to a certain layer
    """
    z = np.dot(w, y_in) + b
    return 1 / (1 + np.exp(-z))

Pay attention on the array's sizes to initialize weighs and biases

In [ ]:
N0 = 2  # input layer size
N1 = 30 # hidden layer size
N2 = 1  # output layer size

# weights and biases
# from input layer to hidden layer:
w1 = np.random.uniform(low=-10, high=+10, size=(N1,N0)) # random weights: N1xN0
b1 = np.random.uniform(low=-1, high=+1, size=N1)        # biases: N1 vector

# weights+biases from hidden layer to output layer:
w2 = np.random.uniform(low=-5, high=+5, size=(N2,N1)) # random weights: N2xN1
b2 = np.random.uniform(low=-1, high=+1, size=N2) # biases

In [ ]:
# evaluate the network by subsequently
# evaluating the two steps (input to hidden and hidden to output)
def apply_basic_net(y_in):
    global w1, b1, w2, b2
    
    y1 = apply_basic_layer(y_in, w1, b1)
    y2 = apply_basic_layer(y1, w2, b2)
    
    return y2

Again, obtain values for a range of inputs

In [ ]:
M = 300  # number of inputs to explore
y_out = np.zeros([M, M])  # array MxM, to hold the result

t0 = time()  # initial time
for j1 in range(M):
    for j2 in range(M):
        value0 = float(j1) / M - 0.5
        value1 = float(j2) / M - 0.5
        y_in = [value0, value1]
        y_out[j1, j2]=apply_basic_net(y_in)

print("Elapsed time: %.0f seconds for %d inputs." % (time()-t0, M**2))

**Note**: Again, this is NOT the most efficient way to do this! (but simple)

In [ ]:
# display image
plt.imshow(y_out, origin='lower', extent=(-0.5,0.5,-0.5,0.5))
plt.colorbar()
plt.title("NN output as a function of input values")
plt.xlabel("y_2")
plt.ylabel("y_1")
plt.show()

Obviously, the shape of the output is more 'complex' that of a simple network without any
hidden layer. Let's go further in that direction...

## 'batch' processing of Neural Networks

Goal: apply a network to many samples in parallel (no 'for' loops!).
Let's solve the less efficient method above.

### Matrix-vector multiplication of array dimensions

See how the dot product works in a no hidden layer NN (or a single step from one layer to the next in a more deep NN).

For instance, let's see with $N_0=8$ (input neurons), $N_1=7$ (output neurons) and $M=50$ (different inputs)

In [ ]:
N0 = 8
N1 = 7
M = 50

W = np.zeros([N1, N0])  # Weight is N1xN0, as before
y = np.zeros([N0, M]) # Let's define a matrix to hold the M different inputs

(np.dot(W, y)).shape  # Let's see what happens

Now, let's try to add the bias vector entries, in the most naive way (beware!)

In [ ]:
B = np.zeros(N1)  # Bias size is corresponding with the output neurons
(np.dot(W, y) + B).shape  # will produce an error!

Why is that error? How can we avoid it?

Ok, we have to think a bit more...

Let's play with the order of the indices (arrays shapes).

Let's try to put the $M$ to the very beginning to hold the different inputs

In [ ]:
y = np.zeros([M, N0])
(np.dot(y, W)).shape  # will produce an error! Why?

We have exploited out earlier... let's think

We can transpose the weight matrix

In [ ]:
W = np.zeros([N0, N1])
(np.dot(y, W)).shape  # Fixed!

Let's add again the bias vector

In [ ]:
B = np.zeros(7)
(np.dot(y, W) + B).shape  # Voila!

This is the desired result, which contains 50 results of 7 values at once. This is full of zeros, but we have asserted that the method is consistent.

### Defining functions to evaluate a layer, and a network with batch processing

Set up for batch processing, i.e. parallel evaluation of many input samples!

In [ ]:
def apply_layer_batch(y_in, w, b): # a function that applies a layer, just as before
    z = np.dot(y_in, w) + b  # but, note different order in matrix product!
    return 1 / (1 + np.exp(-z))

In [ ]:
def apply_net_batch(y_in): # same as before
    global w1, b1, w2, b2
    
    y1 = apply_layer_batch(y_in, w1, b1)
    y2 = apply_layer_batch(y1, w2, b2)
    
    return y2

Let's define the NN model. Note that it is independent of the batch! It just
allows batching with the new ordering of the matrix indices

In [ ]:
N0 = 2  # input layer size
N1 = 30  # hidden layer size
N2 = 1  # output layer size

# from input layer to hidden layer:
w1 = np.random.uniform(low=-10, high=+10, size=(N0,N1)) # NEW ORDER!! N0xN1
b1 = np.random.uniform(low=-1, high=+1, size=N1)        # biases: N1 vector

# from hidden layer to output layer:
w2 = np.random.uniform(low=-5, high=+5, size=(N1, N2)) # NEW ORDER N1xN2
b2 = np.random.uniform(low=-1, high=+1, size=N2) # biases

Let's define an input with a certain batch size (e.g. 10000)

In [ ]:
batchsize = 90000
y = np.random.uniform(low=-1, high=1, size=(batchsize,2))  # Why this 2 here??

In [ ]:
t0 = time()
y_out = apply_net_batch(y)
print("Elapsed time: %f seconds for %d inputs." % (time()-t0, batchsize))

In [ ]:
np.shape(y_out)

These were 90,000 samples evaluated in parallel!!! (less than 1 second)

### Now visualize this multi-layer net, now more efficiently!

Before, we had a two nested loops running all possible values for each input neuron.
Now, we have to think a way to put all that possibilities in a batch array. How to do that?

In [ ]:
M = 300
# Generate a 'mesh grid', i.e. x,y values in an ordered way. Let's see
values = np.linspace(-0.5,0.5,M)  # This crates an array from -0.5 to 0.5 of M elements
v0, v1 = np.meshgrid(values, values)  # Check meshgrid documentation

Let's see what we did:

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(v0, origin='lower')
ax[0].set_title("input value v0")
ax[1].imshow(v1, origin='lower')
ax[1].set_title("input value v1")
plt.show()

Look, `v0` increases with the horizontal direction and remains constant on the
vertical, like the $x$ coordinate. While `v1` runs just on the opposite,
like $y$ coordinate.

In [ ]:
v0flat = v0.flatten()  # make 1D arrays out of 2D array, reading from left to right
v1flat = v1.flatten()  #  then, goes for the next line
# that means: MxM matrix becomes M^2 vector
print('v0flat: ', v0flat, ' ; shape: ', v0flat.shape)
print('v1flat: ', v1flat, ' ; shape: ', v1flat.shape)

In [ ]:
batchsize = v0flat.size  # number of samples
y_in = np.zeros([batchsize,2])  # initialize
y_in[:,0] = v0flat # fill first component (index 0)
y_in[:,1] = v1flat # fill second component

Apply net to all these samples simultaneously! At the end, this is equivalent to the `apply_net` in the double loop.

In [ ]:
t0 = time()
y_out=apply_net_batch(y_in) 
print("Elapsed time: %f seconds for %d inputs." % (time()-t0, batchsize))
y_out.shape

this is not a vector, but a funny flattened matrix of batchsize x 1. Why is `x 1`?

Let's go back from a flattened array to a real matrix

In [ ]:
y_2D = np.reshape(y_out[:,0],[M,M]) 

In [ ]:
plt.imshow(y_2D, origin='lower')
plt.title("NN output (one hidden layer)")
plt.xlabel("v0")
plt.ylabel("v1")
plt.show()

### A network with MANY hidden layers (same size each)

We can create a single `weights` hyper-matrix containing the weight of all hidden layers. It can be seen as a book with several sheets, where every sheet (slice) is a matrix corresponding to a certain weight matrix for that hidden layer. We increase the range of these weight in order to make them more relevant on the NN.

We will do the same for the `biases`. Before, it's a vector, then it becomes a matrix.

In [ ]:
Nlayers = 20  # not counting the input layer & the output layer
LayerSize = 100

# Note the extra dim at first for the weights and biases
weights = np.random.uniform(low=-3, high=3, size=[Nlayers,LayerSize,LayerSize]) 
biases = np.random.uniform(low=-1, high=1, size=[Nlayers,LayerSize])

# for the first layer having 2 inputs
weightsFirst = np.random.uniform(low=-1, high=1, size=[2,LayerSize])
biasesFirst = np.random.uniform(low=-1, high=1, size=LayerSize)

# for the final layer having 1 output
weightsFinal = np.random.uniform(low=-1, high=1, size=[LayerSize,1])
biasesFinal = np.random.uniform(low=-1, high=1, size=1)

In [ ]:
def apply_multi_net(y_in):
    global weights, biases, weightsFinal, biasesFinal, Nlayers
    
    y = apply_layer_batch(y_in, weightsFirst, biasesFirst)    
    for j in range(Nlayers):
        y = apply_layer_batch(y, weights[j,:,:], biases[j,:])  # We take the j-th slice
    output = apply_layer_batch(y, weightsFinal, biasesFinal)
    
    return output

Let's explore a set of pair-values (inputs) using a flattened meshgrid, as before

In [ ]:
# Generate a 'mesh grid', i.e. x,y values in an image
M = 300  # Let's do it quite resolutive
v0, v1 = np.meshgrid(np.linspace(-0.5,0.5,M), np.linspace(-0.5,0.5,M))
batchsize = M**2  # number of samples = M^2  (it's equivalent as before)

y_in = np.zeros([batchsize, 2])  # initialize
y_in[:,0] = v0.flatten() # fill first component (index 0)
y_in[:,1] = v1.flatten() # fill second component

Let's apply the dens NN in batch mode (300 $\times$ 300 = 90,000 different inputs at once)

In [ ]:
t0 = time()
y_out = apply_multi_net(y_in) # apply net to all these samples! (it take some seconds)
print("Elapsed time: %f seconds for %d inputs." % (time()-t0, batchsize))
y_2D = np.reshape(y_out, [M,M]) # back to 2D image

In [ ]:
plt.imshow(y_2D, origin='lower', extent=[-0.5,0.5,-0.5,0.5], interpolation='nearest')
plt.colorbar()
plt.show()

### [Homework] Try with another nonlinear activation function

## Summing up in general functions

Let's define a couple of functions to create an arbitrary deep and fully connected neural network in batch mode.

In [ ]:
def apply_layer(y_in, w, b, activation):
    """ Batch processing of a single layer:
           y_in: input values  -> shape: [batchsize, num_neurons_in]
           w:    weight matrix -> shape: [n_neurons_in, n_neurons_out]
           b:    bias vector   -> length: n_neurons_out
    
           activation is some string of the following ones:
             - sigmoid
             - jump
             - linear
             - reLU
    
           returns the values of the output neurons in the next layer 
              -> shape: [batchsize, n_neurons_out]
    """
    z = np.dot(y_in, w) + b
    if activation == 'sigmoid':
        return 1 / (1 + np.exp(-z))
    elif activation == 'jump':
        return np.array(z>0, dtype='float')
    elif activation == 'linear':
        return z
    elif activation == 'reLU':
        return (z > 0) * z

    
def apply_net(y_in, weights, biases, activations):
    """ Apply a whole network of multiple layers.
          y_in: input values  -> shape: [batchsize, num_neurons_in]
          weights, biases and activations must be any iterable 
          which length is the layers' number containing
              weight matrix  -> shape: [n_neurons_in, n_neurons_out]
              bias vector    -> length: n_neurons_out
              activation str -> sigmoid, jump linear or reLU
          Alternatively, they can be extended matrices
          where a simple slicing generates the proper weight, 
          bias and activation.
    """
    y = y_in
    for j in range(len(biases)):
        y = apply_layer(y, weights[j], biases[j], activations[j])
    return y

## Fancy visualization of Neural Networks with Pure Python

### Some internal routines for plotting the network

In [ ]:
BLUE_COLOR = [0, 0.4, 0.8]
ORANGE_COLOR = [1, 0.3, 0]

def plot_connection_line(ax, X, Y, W, vmax=1.0, linewidth=3):
    """ Draw a fancy line from (X[0], Y[0]) to (X[1], Y[1])
        according to the weight W into the frame ax.
    """
    t = np.linspace(0,1,20)  # free parameter to draw lines
    
    if W > 0:  # Color depending on the weight's sign
        col = BLUE_COLOR
    else:
        col = ORANGE_COLOR
    
    # fancy line from (X0, Y0) to (X1, Y1)
    xx = X[0] + t*(X[1] - X[0])  # Linear in horizontal
    yy = Y[0] + (3*t**2 - 2*t**3) * (Y[1] - Y[0])  # Round borders
    
    # plotting the line according to the weight
    ax.plot(xx, yy, alpha=abs(W)/vmax,
            color=col, linewidth=linewidth)

    
def plot_neuron_alpha(ax, X, Y, B, size=100.0, vmax=1.0):
    """ Draw a single neuron in position (X, Y) according to 
        the bias B, into the frame ax.
    """
    if B > 0:
        col = BLUE_COLOR
    else:
        col = ORANGE_COLOR
        
    ax.scatter([X], [Y], marker='o', c=col, alpha=abs(B)/vmax, 
               s=size, zorder=10)

    
def plot_neuron(ax, X, Y, B, size=100.0, vmax=1.0):
    """ Draw a single neuron in position (X, Y) independently to 
        the bias B, into the frame ax.
    """
    if B > 0:
        col = BLUE_COLOR
    else:
        col = ORANGE_COLOR
        
    ax.scatter([X], [Y], marker='o', c=col, s=size, zorder=10)
    
    
def visualize_network(weights, biases, activations, M=100,
                      y0range=[-1,1], y1range=[-1,1],
                      size=400.0, linewidth=5.0):
    """
    Visualize a neural network with 2 input 
    neurons and 1 output neuron (plot output vs input in a 2D plot)
    
    weights is a list of the weight matrices for the
    layers, where weights[j] is the matrix for the connections
    from layer j to layer j+1 (where j==0 is the input)
    
    weights[j][m,k] is the weight for input neuron k going to output neuron m
    (note: internally, m and k are swapped, see the explanation of
    batch processing in lecture 2)
    
    biases[j] is the vector of bias values for obtaining the neurons 
    in layer j+1, biases[j][k] is the bias for neuron k in layer j+1
    
    activations is a list of the activation functions for
    the different layers: choose 'linear','sigmoid', 
    'jump' (i.e. step-function), and 'reLU'
    
    M is the resolution (MxM grid)
    
    y0range is the range of y0 neuron values (horizontal axis)
    y1range is the range of y1 neuron values (vertical axis)
    """
    # Let's transpose the weight to be able the batch processing
    swapped_weights = []
    for j in range(len(weights)):
        swapped_weights.append(np.transpose(weights[j]))
        
    # Let's create a set of input-pairs by means of a mesh grid
    y0, y1 = np.meshgrid(np.linspace(y0range[0], y0range[1], M),
                         np.linspace(y1range[0], y1range[1], M))
    y_in = np.zeros([M*M, 2])
    y_in[:, 0] = y0.flatten()
    y_in[:, 1] = y1.flatten()
    
    # Let's apply the NN
    y_out = apply_net(y_in, swapped_weights, biases, activations)

    # We will plot a diagram at left and the result at right
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(8,4))
    
    
    # For the diagram
    
    #  1: posX and posY are arrays containing the positions of neurons
    posX = [[0, 0]]  # same column for both (at left)    
    posY = [[-0.5, +0.5]]  # for 2 inputs, let's putted centered in high

    
    vmax = 0.0 # for finding the maximum weight
    vmaxB = 0.0 # for maximum bias
    for j in range(len(biases)):  # for every layer on the NN
        n_neurons = len(biases[j])  # neurons in the current layer
        
        posX.append(np.full(n_neurons, j+1))  # next column to the previous one
        posY.append(np.array(range(n_neurons)) - 0.5 * (n_neurons-1)) # spread
        
        vmax = np.maximum(vmax, np.max(np.abs(weights[j])))  # to get the maximum
        vmaxB = np.maximum(vmaxB, np.max(np.abs(biases[j])))

    #   2: plot connections
    for j in range(len(biases)):  # for each layer
        for k in range(len(posX[j])):  # for each neuron
            for m in range(len(posX[j+1])):  # for each following neuron
                plot_connection_line(ax[0],  # first column of the plot
                                     [posX[j][k], posX[j+1][m]], # [X0,X1]
                                     [posY[j][k], posY[j+1][m]], # [Y0,Y1]
                                     swapped_weights[j][k,m],    # its weight
                                     vmax=vmax,  # to get normalized plots
                                     linewidth=linewidth)
    
    #   3: plot neurons
    for k in range(len(posX[0])):  # input neurons (have no bias!)
        plot_neuron(ax[0], posX[0][k], posY[0][k],
                    vmaxB, vmax=vmaxB, size=size)
        
    for j in range(len(biases)): # all other neurons
        for k in range(len(posX[j+1])):
            plot_neuron_alpha(ax[0], posX[j+1][k], posY[j+1][k],
                              biases[j][k], vmax=vmaxB, size=size)
    
    ax[0].axis('off')
    
    # now: the output of the network
    img = ax[1].imshow(np.reshape(y_out, [M,M]),
                       origin='lower',
                       extent=[y0range[0],y0range[1],y1range[0],y1range[1]])
    ax[1].set_xlabel(r'$y_0$')
    ax[1].set_ylabel(r'$y_1$')
    
    axins1 = inset_axes(ax[1],
                        width="40%",  # width = 50% of parent_bbox width
                        height="5%",  # height : 5%
                        loc='upper right')

    imgmin = np.min(y_out)
    imgmax = np.max(y_out)
    color_bar = fig.colorbar(img, cax=axins1, orientation="horizontal",
                             ticks=np.linspace(imgmin,imgmax,3))
    cbxtick_obj = plt.getp(color_bar.ax.axes, 'xticklabels')
    plt.setp(cbxtick_obj, color="white")
    axins1.xaxis.set_ticks_position("bottom")

    ax[1].set_title(activations[0])
    
    plt.show()

### No hidden layer NN

Let's visualize a simple network (no hidden layer) with different activation function.

Notice that no hidden layer means that weight, biases and activations are list of length 1

In [ ]:
RANGE = [-3, 3]  # all tests will be with the same range

weights=[ [     # a list of matrices (length 1 in this case)
    [0.2, 0.9]  # from 2 input neurons to a single output neuron: 1x2
    ] ]

biases=[   # a list of vectors (length 1 in this case)
    [0.5]  # bias for 1 single output neuron: 1 value
    ]

visualize_network(weights, biases, ['sigmoid'],
                  y0range=RANGE, y1range=RANGE)

visualize_network(weights, biases, ['jump'],
                  y0range=RANGE, y1range=RANGE)

### Deep dense NN

Let's visualize a NN of 1 hidden layer of 3 neurons using different activation functions

In [ ]:
weights = [ [  # a list of matrices (length 2 in this case)
        [0.2, 0.9],  # hidden layer of 3 neurons
        [-0.5, 0.3], #  -> 3 neurons with 2 inputs: matrix of 3x2
        [0.8, -1.3]
    ],                 
    [ 
        [-0.3,0.7,0.5] # from 3 hidden neurons to one output: 1x3
    ]  ]

biases=[  # a list of vectors (length 2 in this case)
        [0.1, -0.5, -0.5], # biases of 3 hidden neurons
        [-0.2] # bias for output neuron
       ]

visualize_network(weights, biases, ['sigmoid', 'sigmoid'],
                  y0range=RANGE, y1range=RANGE)

visualize_network(weights, biases, ['jump', 'jump'],
                  y0range=RANGE, y1range=RANGE)

visualize_network(weights, biases, ['reLU', 'reLU'],
                  y0range=RANGE, y1range=RANGE)

visualize_network(weights, biases, ['linear', 'linear'],
                  y0range=RANGE, y1range=RANGE)

More layers and some activation function combination

In [ ]:
# now with 5 intermediate neurons, for fun:

weights = [ [  # list of matrices
                [0.2, 0.9],  # 5x2 
                [-0.5, 0.3], 
                [0.8, -1.3], 
                [-0.3, -0.9], 
                [-0.8, -1.2] 
          ],
          [ 
                [-0.3, 0.7, 0.5, -0.3, 0.4]  # 1x5
          ] ]

biases = [  # list of vectors
                [0.1, -0.5, -0.5, 0.3, 0.2],  # bias for the hidden layer
                [0.5]  # bias for the output bias
         ]   # bias for the output neuron

visualize_network(weights, biases,              
                  activations=[ 'jump', 'linear' ],
                  y0range=RANGE, y1range=RANGE, M=400)

visualize_network(weights, biases,              
                  activations=[ 'sigmoid', 'linear' ],
                  y0range=RANGE, y1range=RANGE, M=400)

Let's sharper the sigmoid: scale all weights and biases!

In [ ]:
factor=10.0  # as high as sharpen 

# this needs np.array(), because you cannot do factor*<python-list>
ws = [factor*np.array(matrix) for matrix in weights]
bs = [factor*np.array(vector) for vector in biases]

visualize_network(ws, bs, activations=[ 'sigmoid', 'linear' ],
                  y0range=RANGE, y1range=RANGE, M=400)

### Something not random

Many superimposed lines can be used to construct arbitrary shapes, with only a single hidden layer.

Let's draw a kind of circle by means of several lines.

In [ ]:
factor = 10  # factor to sharpen sigmoid

n_lines = 10  # number of lines to draw

phi = np.linspace(0, 2*np.pi, n_lines+1)  # Angular variable
phi = phi[0:-1]  # the last value is 2pi, which is equivalent to the 0

weight_hidden = np.zeros([n_lines, 2])   # comment this shape
weight_hidden[:,0] = factor*np.cos(phi)  # x=cos(phi)
weight_hidden[:,1] = factor*np.sin(phi)  # y=sin(phi)

bias_hidden = np.full(n_lines, factor*(+0.5))  # all neurons acts equally


visualize_network(weights=[ 
                            weight_hidden,           # from input to hidden
                            np.full([1,n_lines],1.0) # from hidden to output
                          ],
                  biases=[ 
                            bias_hidden,
                            [0.0]
                         ],
                  activations=['sigmoid',  # activation for hidden
                               'linear'    # activation for output
                              ],
                  y0range=RANGE, y1range=RANGE,
                  size=30.0, M=400)